In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai import *
import numpy as np

In [3]:
from exp.nb_01 import load_data
from exp.nb_02 import gini_score

In [4]:
train_df, test_df = load_data(keep_df=True)
n = len(train_df)
valid_ids = list(range(int(n * .8),n))
train_df.iloc[valid_ids]['label'].sum()

/home/ubuntu/anaconda3/envs/thanos/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/home/ubuntu/anaconda3/envs/thanos/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


103

In [5]:
train_df.head()

,id,label,province,district,age_source1,age_source2,maCv,FIELD_1,FIELD_2,FIELD_3,...,FIELD_48,FIELD_49,FIELD_50,FIELD_51,FIELD_52,FIELD_53,FIELD_54,FIELD_55,FIELD_56,FIELD_57
0,0,0,NaN,NaN,NaN,NaN,NaN,1,1.0,2547.0,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,Tỉnh Đồng Nai,Thành phố Biên Hòa,44.0,44.0,None,1,0.0,-1.0,...,False,True,29.77,17.561,30.955,31.171,0.0,0.04,0.0,0.0
2,2,0,Tỉnh Đồng Nai,Huyện Long Thành,30.0,30.0,Công nhân,0,1.0,3273.0,...,True,True,29.77,56.512,30.955,31.171,0.0,0.16,0.0,0.0
3,3,0,Tỉnh Tuyên Quang,Thành phố Tuyên Quang,43.0,NaN,NaN,0,1.0,3991.0,...,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,Thành phố Hồ Chí Minh,Quận 1,21.0,21.0,None,0,1.0,1450.0,...,True,True,29.77,13.789,30.955,31.171,0.0,0.03,0.0,0.0


In [6]:
del train_df['id']

In [7]:
dep_var = 'label'
#cat_names = data.select_dtypes(exclude=['int', 'float']).columns
cat_names = list(train_df.select_dtypes(exclude=['int', 'float']).columns)
#cont_names = data.select_dtypes([np.number]).columns
cont_names = list(train_df.select_dtypes([np.number]).columns[1:])

In [8]:
cat_names[0:6]

['province', 'district', 'maCv', 'FIELD_7', 'FIELD_8', 'FIELD_9']

In [9]:
train_df = train_df[cat_names + cont_names + [dep_var]]

In [10]:
from fastai.tabular import FillMissing, Categorify, Normalize
procs = [FillMissing, Categorify, Normalize]

In [11]:
#export
from fastai.tabular import TabularList
def load_data_bunch(train_df, test_df, cat_names, cont_names, procs, dep_var = 'label', valid_ratio=0.8):
    n = len(train_df)
    valid_ids = list(range(int(n * valid_ratio),n))
    test = TabularList.from_df(test_df, cat_names=cat_names, cont_names=cont_names, procs=procs)
    data = (TabularList.from_df(train_df, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                        .split_by_idx(valid_ids)
                        .label_from_df(cols = dep_var)
                        .add_test(test, label=0)
                        .databunch())
    return data

In [12]:
# Test Tabular List
test_id = test_df['id']
# Train Data Bunch
data = load_data_bunch(train_df, test_df, cat_names, cont_names, procs)

data.show_batch(rows=3)

province,district,maCv,FIELD_7,FIELD_8,FIELD_9,FIELD_10,FIELD_11,FIELD_12,FIELD_13,FIELD_17,FIELD_18,FIELD_19,FIELD_20,FIELD_23,FIELD_24,FIELD_25,FIELD_26,FIELD_27,FIELD_28,FIELD_29,FIELD_30,FIELD_31,FIELD_35,FIELD_36,FIELD_37,FIELD_38,FIELD_39,FIELD_40,FIELD_41,FIELD_42,FIELD_43,FIELD_44,FIELD_45,FIELD_47,FIELD_48,FIELD_49,age_source1_na,age_source2_na,FIELD_2_na,FIELD_3_na,FIELD_4_na,FIELD_5_na,FIELD_6_na,FIELD_16_na,FIELD_21_na,FIELD_22_na,FIELD_50_na,FIELD_51_na,FIELD_52_na,FIELD_53_na,FIELD_54_na,FIELD_55_na,FIELD_56_na,FIELD_57_na,age_source1,age_source2,FIELD_1,FIELD_2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_14,FIELD_15,FIELD_16,FIELD_21,FIELD_22,FIELD_32,FIELD_33,FIELD_34,FIELD_46,FIELD_50,FIELD_51,FIELD_52,FIELD_53,FIELD_54,FIELD_55,FIELD_56,FIELD_57,target
Tỉnh Đồng Tháp,Huyện Tân Hồng,#na#,"['DN', 'HS']",#na#,na,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,True,True,True,False,True,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,1.8270,-0.1685,0.5377,0.5513,0.8212,0.4485,-0.3875,-0.3145,1.2271,1.2343,-0.2477,-0.0061,-0.2612,1.3960,0.6815,1.0942,-0.8197,-0.3003,-0.0022,-0.2454,-0.2354,-0.2564,0.0631,-0.2145,-0.2078,0
Tỉnh Bình Phước,Huyện Hớn Quản,CN - L3. CCBI,"['BT', 'HS']",MALE,DN,GH,12,0,YN,None,False,False,False,True,None,True,True,True,True,FALSE,TRUE,FALSE,Four,FALSE,True,False,VN,None,I,Zezo,None,Two,2,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-1.3032,-1.2278,0.5377,0.5513,-0.4305,0.4485,-0.3875,-0.3145,-0.8149,-0.8101,-0.2477,-0.0061,-0.2401,-0.7163,0.6815,-0.9138,1.2199,-0.3003,1.7318,-0.2454,-0.2354,-0.2564,1.1874,-0.2145,-0.2078,0
#na#,#na#,Công nhân,[],FEMALE,GD,T1,12,0,BI,None,True,False,False,True,None,True,False,True,True,FALSE,TRUE,FALSE,Zero,FALSE,TRUE,False,VN,None,V,Zezo,None,One,1,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.1650,1.9501,0.5377,-1.8137,-1.1822,-0.5854,-0.8886,-0.3145,-0.8149,-0.8101,-0.2477,-0.0061,1.6339,-0.7163,0.6815,-0.9138,1.2199,-0.3003,-0.4907,-0.2454,-0.2354,-0.2564,-0.7801,-0.2145,-0.2078,1


In [13]:
#export
from fastai.tabular import tabular_learner
def load_learner(data, params): return tabular_learner(data, **params, metrics=accuracy, callback_fns=ShowGraph)

In [14]:
params = {'layers': [500, 300],
          'ps': [0.002,0.02],
          'emb_drop': 0.1,
          'emb_szs':{'province': 5,
                     'district': 4,
                     'maCv':8,
                     'FIELD_7': 6},
         }
learn = load_learner(data, params)

NameError: name 'accuracy' is not defined

In [ ]:
learn.model

In [ ]:
# select the appropriate learning rate
learn.lr_find()

In [ ]:
# we typically find the point where the slope is steepest
learn.recorder.plot()

In [ ]:
# Fit the model based on selected learning rate
learn.fit_one_cycle(1, max_lr=slice(1e-03))

In [ ]:
# Analyse our model
learn.model
learn.recorder.plot_losses()

In [ ]:
# Predict our target value
predictions, *_ = learn.get_preds(DatasetType.Valid)
y_score = np.array(predictions[:,1].tolist())
y_valid = list(train_df.iloc[valid_ids].label)
gini_score(y_valid, y_score)

In [ ]:
# Predict our target value
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.array(predictions[:,1].tolist())
# create submission file to submit in Kaggle competition
submission = pd.DataFrame({'id': test_id, 'label': labels})
submission.to_csv('data/submission.csv', index=False)
submission.head()